<a href="https://colab.research.google.com/github/nahbos/AUT-Neural-Networks/blob/main/HW08/HW08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Sobhan Moradian Daghigh
- 7/12/2022
- ANN - HW08


In [3]:
# !pip install -U 'tensorflow-text==2.8.*'

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_text
import time

# Load data

In [9]:
dataset = tfds.load('ted_hrlr_translate/pt_to_en', as_supervised=True)
x_train, x_test, x_val = dataset['train'], dataset['test'], dataset['validation']

## Preview

In [37]:
for pt_sentences, en_sentences in x_train.batch(5).take(1):
  for pt, en in zip(pt_sentences.numpy(), en_sentences.numpy()):
    print('Portuguese: [{}]'.format(pt.decode('utf-8')))
    print('English:    [{}]'.format(en.decode('utf-8')))
    print('---------------------------------')

Portuguese: [e quando melhoramos a procura , tiramos a única vantagem da impressão , que é a serendipidade .]
English:    [and when you improve searchability , you actually take away the one advantage of print , which is serendipity .]
---------------------------------
Portuguese: [mas e se estes fatores fossem ativos ?]
English:    [but what if it were active ?]
---------------------------------
Portuguese: [mas eles não tinham a curiosidade de me testar .]
English:    [but they did n't test for curiosity .]
---------------------------------
Portuguese: [e esta rebeldia consciente é a razão pela qual eu , como agnóstica , posso ainda ter fé .]
English:    [and this conscious defiance is why i , as an agnostic , can still have faith .]
---------------------------------
Portuguese: [`` `` '' podem usar tudo sobre a mesa no meu corpo . '']
English:    [you can use everything on the table on me .]
---------------------------------


# Text tokenization
Downloading a model which was implemented for tokenizing the sentences.

In [ ]:
tf.keras.utils.get_file(
    f'{model_name}.zip',
    f'https://storage.googleapis.com/download.tensorflow.org/models/{'ted_hrlr_translate_pt_en_converter'}.zip',
    cache_dir='.', cache_subdir='', extract=True)